In [1]:
#Importing Libraries
import pandas as pd
import os
import numpy as np
os.chdir('C:/Users/sinha/OneDrive/Desktop/Learning/Analysis/Accelya/bsp-anon')

In [2]:
#Reading file

df = pd.read_csv('bsp-anon.csv')

C:\Users\sinha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Set maximum columns display
pd.set_option('display.max_columns', 100)

#checking head
df.head()

,bks24_pnrr,bar64_tdnr,airlinecode,bks24_agtn,bks30_tdam,curcode,bks24_dais,bks24_todc,bki63_orac,bki63_dstc,bki63_carr,bki63_ftnr,bki63_ftda,bki63_ftdt,bki63_cabi,bki63_rbkd,bks39_rdii,bki63_segi,bki63_stpo,bks24_rfic,n_tktt,n_segments,label,label_amount
0,PWNINX,6.277008e+10,12,3846899,3095.95,BRL,04/04/2018,GPOGPO,GPO,BSB,JP,1697,01-Nov,1820,NaN,N,NaN,1,NaN,NaN,1,6,0,0.0
1,6Q9T6M,9.855730e+10,12,79466150,632.18,CAD,05/04/2018,STLSTL,SKA,STL,L9,5288,14-Apr,128P,NaN,G,NaN,4,NaN,NaN,1,4,0,0.0
2,0WLXLN,4.920000e+11,3,79466150,722.84,CAD,24/11/2018,PVRPVR,ORF,PVR,UE,1802,15-Mar,420P,NaN,K,NaN,1,NaN,NaN,1,4,0,0.0
3,A15N5F,5.560000e+11,3,849146,787.17,CAD,22/06/2018,GDLGDL,ILF,MTY,EZ,2467,02-Mar,842,NaN,E,NaN,4,X,NaN,1,6,0,0.0
4,CY92ZY,3.505397e+10,37,79466150,2132.73,CAD,30/11/2017,SDFSDF,GBE,LPA,V4,8358,02-Sep,1105P,NaN,N,NaN,4,X,NaN,1,5,0,0.0


In [4]:
#Checking null values and dtype

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308588 entries, 0 to 308587
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   bks24_pnrr    308588 non-null  object 
 1   bar64_tdnr    308588 non-null  float64
 2   airlinecode   308588 non-null  int64  
 3   bks24_agtn    308588 non-null  int64  
 4   bks30_tdam    308588 non-null  float64
 5   curcode       308588 non-null  object 
 6   bks24_dais    308588 non-null  object 
 7   bks24_todc    308588 non-null  object 
 8   bki63_orac    308588 non-null  object 
 9   bki63_dstc    308588 non-null  object 
 10  bki63_carr    308588 non-null  object 
 11  bki63_ftnr    308588 non-null  object 
 12  bki63_ftda    308588 non-null  object 
 13  bki63_ftdt    308588 non-null  object 
 14  bki63_cabi    538 non-null     float64
 15  bki63_rbkd    308588 non-null  object 
 16  bks39_rdii    77691 non-null   object 
 17  bki63_segi    308588 non-null  int64  
 18  bki6

In [12]:
#Handle missing value by filling either NA or O as defined by the IATA guideline

df['bki63_cabi'].fillna('NA', inplace = True)
df['bks39_rdii'].fillna('NA', inplace = True)
df['bki63_stpo'].fillna('O', inplace = True)
df['bks24_rfic'].fillna('NA', inplace = True)

In [13]:
df.isnull().sum()

bks24_pnrr                  0
bar64_tdnr                  0
airlinecode                 0
bks24_agtn                  0
bks30_tdam                  0
curcode                     0
bks24_dais                  0
bks24_todc                  0
bki63_orac                  0
bki63_dstc                  0
bki63_carr                  0
bki63_ftnr                  0
bki63_ftda                  0
bki63_ftdt                  0
bki63_cabi                  0
bki63_rbkd                  0
bks39_rdii                  0
bki63_segi                  0
bki63_stpo                  0
bks24_rfic                  0
n_tktt                      0
n_segments                  0
label                       0
label_amount                0
Dollar_Conversion           0
Amount(USD)                 0
booking_travel_days_diff    0
departure_time              0
journey_strng               0
dtype: int64

In [5]:
#Reading currency conversion document to convert currency which was available. Although there are libraries for conversion
curr = pd.read_csv('currency con.csv')
curr.head()

,curcode,Dollar_Conversion
0,CAD,0.700
1,EUR,1.080
2,BRL,0.180
3,GBP,1.230
4,MXN,0.041


In [6]:
#Merge two datasets and create a column with USD
df = pd.merge(df, curr, on = ['curcode'])
df['Amount(USD)'] = df['bks30_tdam'] * df['Dollar_Conversion']

In [7]:
#Departure Dates are without year, adding year based on logic if departure date is before booking date, then departure date should be of next year 

df['bki63_ftda'] = pd.to_datetime(df['bki63_ftda'], format='%d-%b').dt.strftime('%m-%d')
df['bks24_dais'] = pd.to_datetime(df['bks24_dais'])
df['bki63_ftda'] = pd.to_datetime(pd.Series([''.join([str(x[1].year),'-',x[0]]) for x in zip(df['bki63_ftda'], df['bks24_dais'])]))
df['bki63_ftda'] = np.where((df['bki63_ftda'] - df['bks24_dais']) < '0 days' ,df['bki63_ftda'] + pd.offsets.DateOffset(years=1), df['bki63_ftda'])

#Calculate difference between booking and departure date

df['booking_travel_days_diff'] = df['bki63_ftda'] - df['bks24_dais']

In [9]:
#Define function to handle inconsistency in time format (alphannumeric, 24Hrs, 12 hrs etc)

def time(row):
    
    text = row['bki63_ftdt']
    
    if type(text) == int:
        
        if text < 60:
            text = pd.to_datetime(text, format='%M').strftime('%H:%M')
        
        elif text >= 100 and text < 130:
            hr = str(text)[0]
            Mn = str(text)[1:]
            Time = '0' + hr + ':' + Mn
            text = pd.to_datetime(Time, format='%H:%M').strftime('%H:%M')
            
        elif text >= 130 and text < 1200:
            text = pd.to_datetime(text, format='%I%M').strftime('%H:%M')
        
        elif text >=1200 and text <= 2359:
            text = pd.to_datetime(text, format='%H%M').strftime('%H:%M')         
            
        
    elif type(text) == str:
        
        counter_P = text.find('P')
        
        if counter_P > 0:
            text = text.replace('P', '')
            text = int(text)
            
            if text >= 1200 and text < 1300:
                text = text
            else:
                text = text + 1200
        else:
            text = text.replace('N', '')
            text = text.replace('A', '')
            text = int(text)
            
        if text < 60:
            text = pd.to_datetime(text, format='%M').strftime('%H:%M')
        
        elif text >= 100 and text < 130:
            hr = str(text)[0]
            Mn = str(text)[1:]
            Time = '0' + hr + ':' + Mn
            text = pd.to_datetime(Time, format='%H:%M').strftime('%H:%M')
            
        elif text >= 130 and text < 1200:
            text = pd.to_datetime(text, format='%I%M').strftime('%H:%M')
        
        elif text >=1200 and text <= 2359:
            text = pd.to_datetime(text, format='%H%M').strftime('%H:%M')
            
    
    return pd.to_datetime(text, format='%H:%M', utc=True)
        

In [10]:
#Apply the function on dataframe

df['departure_time'] = df.apply(time, axis=1)

In [11]:
#Join origin and departure string

df['journey_strng'] = df['bki63_orac'] + df['bki63_dstc']

In [14]:
#Bucketize departure time as 'Late Night', 'Early Morning','Morning','Noon','Evening','Night'

df['departure_hour'] = df['departure_time'].dt.hour
bucket = [0,4,8,12,16,20,24]
label = ['Late Night', 'Early Morning','Morning','Noon','Evening','Night']
df['departure_type'] = pd.cut(df['departure_hour'], bins=bucket, labels=label)

In [51]:
#Check if all PNRs have unique or single booking date 

check = df.groupby(by='bks24_pnrr', as_index=False).agg({'bks24_dais': pd.Series.nunique})

In [57]:
check[check['bks24_dais'] > 1]

,bks24_pnrr,bks24_dais
431,059ALI,2
680,07V9CN,2
1614,0HXPET,2
1732,0JALKJ,2
3759,173N7B,2
...,...,...
111905,Z2RLWQ,2
113263,ZHZVDD,2
113622,ZM73CQ,2
113664,ZMP2OI,2


In [60]:
df[df['bks24_pnrr'] == '059ALI']

,bks24_pnrr,bar64_tdnr,airlinecode,bks24_agtn,bks30_tdam,curcode,bks24_dais,bks24_todc,bki63_orac,bki63_dstc,bki63_carr,bki63_ftnr,bki63_ftda,bki63_ftdt,bki63_cabi,bki63_rbkd,bks39_rdii,bki63_segi,bki63_stpo,bks24_rfic,n_tktt,n_segments,label,label_amount,Dollar_Conversion,Amount(USD),booking_travel_days_diff,departure_time,journey_strng,departure_hour,departure_type
71960,059ALI,6.997128e+10,37,85630581,0.00,CAD,2018-10-17,MHTMHT,MHT,CHA,BF,9690,2018-10-27,1340,NA,T,NA,1,O,NA,2,2,0,0.0,0.7,0.000,10 days,1900-01-01 13:40:00+00:00,MHTCHA,13,Noon
138402,059ALI,6.997128e+10,37,85630581,0.00,CAD,2018-10-17,MHTMHT,CHA,MHT,BF,1045,2018-10-29,1125,NA,T,NA,2,O,NA,2,2,0,0.0,0.7,0.000,12 days,1900-01-01 11:25:00+00:00,CHAMHT,11,Morning
141933,059ALI,6.997128e+10,37,85630581,713.05,CAD,2018-04-10,MHTMHT,MHT,CHA,BF,9690,2018-10-27,1340,NA,T,NA,1,O,NA,2,2,0,0.0,0.7,499.135,200 days,1900-01-01 13:40:00+00:00,MHTCHA,13,Noon
152938,059ALI,6.997128e+10,37,85630581,713.05,CAD,2018-04-10,MHTMHT,CHA,MHT,BF,1045,2018-10-29,1125,NA,T,NA,2,O,NA,2,2,0,0.0,0.7,499.135,202 days,1900-01-01 11:25:00+00:00,CHAMHT,11,Morning


In [70]:
check = df.groupby(by='bks24_pnrr', as_index=False).agg({'bki63_segi': pd.Series.nunique})

In [72]:
#drop unnecessary columns

df.drop(['bar64_tdnr', 'bks30_tdam', 'bks24_agtn', 'bki63_ftnr', 'bki63_ftdt', 'Dollar_Conversion'], axis = 1, inplace = True)

In [73]:
#Save the clean file

df.to_csv('Clean_File.csv')